# Preparation

Load necessary modules and initialize relavent devices

In [1]:
%run seis-idd/tomo_init.py

/Users/chenzhang/opt/anaconda3/envs/bluesky/lib/python3.7/site-packages/databroker/_core.py:962: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)


You can check the __prefined variables and functions__ in the following two dictionaries:

```
keywords_vars = {}  # {name: short description}
keywords_func = {}  # {name: short descciption}
```

or the corresponding functions:

```
list_predefined_vars()
list_predefined_func()
```

In [3]:
list_predefined_vars()
list_predefined_func()

name,description
HOSTNAME,host name
USERNAME,PI/user name
metadata_db,Default metadata handler
RE,Default RunEngine instance


name,description
getRunEngine,Get a bluesky RunEngine
load_config,load yaml to a dict


Please fill out the experiment details below

In [ ]:
RE.md['beamline_id']      = 'APS 6-ID-D'
RE.md['proposal_id']      = 'internal test'
RE.md['pid']              = os.getpid()
RE.md['login_id']         = USERNAME + '@' + HOSTNAME
RE.md['BLUESKY_VERSION']  = bluesky.__version__
RE.md['OPHYD_VERSION']    = ophyd.__version__
RE.md['apstools_VERSION'] = apstools.__version__
RE.md['SESSION_STARTED']  = datetime.isoformat(datetime.now(), " ")

In [ ]:
list_predefined_vars()

# Initialization

Perform control initialization for all related hardware.